# Fire stats table

## Goal
Take the raw data created by fire_stats_data and turn it into a summary table for the appendix that shows the area of high severity fire in the region, in LSRs, and in old-growth in LSRs by year group (and total of year groups), stratified by ecoregion into Coast Range, Olympic Peninsula, OR Western Cascades, WA Western Cascades, and a total of ecoregions. 

In [1]:
import pandas as pd
import os
import numpy as np

In [2]:
from datetime import date
today = date.today().strftime(format="%Y%m%d")

In [36]:
df = pd.read_csv("fire_stats_data_20210802.csv")

## Processing

Subset the data to only include high severity (>75% mortality).

In [4]:
high = df[df.severity.isin(["High", "Very high"])].copy()

Create a new column for grouping by ecoregion that splits the Cascades into OR and WA

In [5]:
high["eco_group"] = high.apply(lambda x : " ".join([x.state, x.ecoregion]) if x.ecoregion == "Western Cascades" else x.ecoregion, axis=1)

### Ecoregion

Group and summarize the data to get the area and percent burned in each ecoregion

In [6]:
ecoregion_table = (
    high
    .groupby(["eco_group", "yr_group"])
    .agg({"hectares_burned": sum, "region_hectares": "first"})
)

Aggregate the data to get the total for both year groups

In [7]:
ecoregion_total = ecoregion_table.groupby(["eco_group"]).agg({"hectares_burned": sum, "region_hectares": "first"})

Combine the grouped data and total data.

In [8]:
ecoregion_total["yr_group"] = "Total"
ecoregion_total = ecoregion_total.groupby(["eco_group", "yr_group"]).sum()
ecoregion_combined = pd.concat([ecoregion_table, ecoregion_total])
ecoregion_data = ecoregion_combined.groupby(["eco_group", "yr_group"]).sum()

### LSRs

Group and summarize the data to get the area and percent burned in LSRs

In [9]:
lsr_table = (
    high
    .groupby(["eco_group", "yr_group"])
    .agg({"lsr_hectares_burned": sum, "region_lsr_hectares": "first"})
)

Aggregate the data to get the total for both year groups

In [10]:
lsr_total = lsr_table.groupby(["eco_group"]).agg({"lsr_hectares_burned": sum, "region_lsr_hectares": "first"})

Combine the grouped data and total data.

In [11]:
lsr_total["yr_group"] = "Total"
lsr_total = lsr_total.groupby(["eco_group", "yr_group"]).sum()
lsr_combined = pd.concat([lsr_table, lsr_total])
lsr_data = lsr_combined.groupby(["eco_group", "yr_group"]).sum()

In [12]:
lsr_data

lsr_hectares_burned  \
eco_group                   yr_group                           
Coast Range                 1985 - 2019               635.13   
                            2020                        0.63   
                            Total                     635.76   
Olympic Peninsula           1985 - 2019               334.53   
                            2020                        0.00   
                            Total                     334.53   
Oregon Western Cascades     1985 - 2019             24235.92   
                            2020                    18283.50   
                            Total                   42519.42   
Washington Western Cascades 1985 - 2019              3534.39   
                            2020                     2307.33   
                            Total                    5841.72   

                                         region_lsr_hectares  
eco_group                   yr_group                          
Coast Range                 1985 - 2019        353348.443324  
                            2020               353348.443324  
                            Total              353348.443324  
Olympic Peninsula           1985 - 2019        149682.150101  
                            2020               149682.150101  
                            Total              149682.150101  
Oregon Western Cascades     1985 - 2019        539256.535473  
                            2020               539256.535473  
                            Total              539256.535473  
Washington Western Cascades 1985 - 2019        467968.446069  
                            2020               467968.446069  
                            Total              467968.446069

### OG in LSRs

Group and summarize the data to get the area and percent of old growth burned in LSRs

In [13]:
lsr_og_table = (
    high[high.struccond.isin(['Large tree - moderate/closed', 'Large/giant tree - moderate/closed'])]
    .groupby(["eco_group", "yr_group"])
    .agg({"lsr_hectares_burned": sum, "region_lsr_og_hectares": "first"})
)
lsr_og_table = lsr_og_table.rename({"lsr_hectares_burned": "lsr_og_hectares_burned"}, axis=1)

Aggregate the data to get the total for both year groups

In [14]:
lsr_og_total = lsr_og_table.groupby(["eco_group"]).agg({"lsr_og_hectares_burned": sum, "region_lsr_og_hectares": "first"})

Combine the grouped data and total data.

In [15]:
lsr_og_total["yr_group"] = "Total"
lsr_og_total = lsr_og_total.groupby(["eco_group", "yr_group"]).sum()
lsr_og_combined = pd.concat([lsr_og_table, lsr_og_total])
lsr_og_data = lsr_og_combined.groupby(["eco_group", "yr_group"]).sum()

### Combining Ecoregion, LSRs, and OG LSRs

In [16]:
all_data = pd.concat([ecoregion_data, lsr_data, lsr_og_data], axis=1)

Aggregate all of the ecoregions together into a "Region" group

In [17]:
region_data = all_data.groupby("yr_group").sum()
region_data["eco_group"] = "Region"
region_data = region_data.groupby(["eco_group", "yr_group"]).sum()

Calculate the percent of each group burned in the "Region" group

In [18]:
region_data["percent_burned"] = region_data.hectares_burned.divide(region_data.region_hectares).multiply(100)
region_data["lsr_percent_burned"] = region_data.lsr_hectares_burned.divide(region_data.region_lsr_hectares).multiply(100)
region_data["lsr_og_percent_burned"] = region_data.lsr_og_hectares_burned.divide(region_data.region_lsr_og_hectares).multiply(100)

Calculate the percent of each group burned in each ecoregion

In [19]:
all_data["percent_burned"] = all_data.hectares_burned.divide(all_data.region_hectares).multiply(100)
all_data["lsr_percent_burned"] = all_data.lsr_hectares_burned.divide(all_data.region_lsr_hectares).multiply(100)
all_data["lsr_og_percent_burned"] = all_data.lsr_og_hectares_burned.divide(all_data.region_lsr_og_hectares).multiply(100)

In [20]:
final_df = pd.concat([region_data, all_data])

In [21]:
final_df

hectares_burned  region_hectares  \
eco_group                   yr_group                                        
Region                      1985 - 2019        145480.23     1.025124e+07   
                            2020               177136.92     1.025124e+07   
                            Total              322617.15     1.025124e+07   
Coast Range                 1985 - 2019          1756.98     2.358035e+06   
                            2020                  169.20     2.358035e+06   
                            Total                1926.18     2.358035e+06   
Olympic Peninsula           1985 - 2019           713.88     1.733874e+06   
                            2020                    0.54     1.733874e+06   
                            Total                 714.42     1.733874e+06   
Oregon Western Cascades     1985 - 2019        112970.79     2.894037e+06   
                            2020               174211.29     2.894037e+06   
                            Total              287182.08     2.894037e+06   
Washington Western Cascades 1985 - 2019         30038.58     3.265294e+06   
                            2020                 2755.89     3.265294e+06   
                            Total               32794.47     3.265294e+06   

                                         lsr_hectares_burned  \
eco_group                   yr_group                           
Region                      1985 - 2019             28739.97   
                            2020                    20591.46   
                            Total                   49331.43   
Coast Range                 1985 - 2019               635.13   
                            2020                        0.63   
                            Total                     635.76   
Olympic Peninsula           1985 - 2019               334.53   
                            2020                        0.00   
                            Total                     334.53   
Oregon Western Cascades     1985 - 2019             24235.92   
                            2020                    18283.50   
                            Total                   42519.42   
Washington Western Cascades 1985 - 2019              3534.39   
                            2020                     2307.33   
                            Total                    5841.72   

                                         region_lsr_hectares  \
eco_group                   yr_group                           
Region                      1985 - 2019         1.510256e+06   
                            2020                1.510256e+06   
                            Total               1.510256e+06   
Coast Range                 1985 - 2019         3.533484e+05   
                            2020                3.533484e+05   
                            Total               3.533484e+05   
Olympic Peninsula           1985 - 2019         1.496822e+05   
                            2020                1.496822e+05   
                            Total               1.496822e+05   
Oregon Western Cascades     1985 - 2019         5.392565e+05   
                            2020                5.392565e+05   
                            Total               5.392565e+05   
Washington Western Cascades 1985 - 2019         4.679684e+05   
                            2020                4.679684e+05   
                            Total               4.679684e+05   

                                         lsr_og_hectares_burned  \
eco_group                   yr_group                              
Region                      1985 - 2019                10921.68   
                            2020                        9232.38   
                            Total                      20154.06   
Coast Range                 1985 - 2019                  285.66   
                            2020                           0.36   
                            Total                        286.02   
Olympic Peninsula

## Formatting

Format the final data table for the paper

In [22]:
print_df = final_df.copy()

print_df["Region/ecoregion"] = print_df.apply(lambda x : f"{int(x.hectares_burned):,} ha ({x.percent_burned:0.1f}%)", axis=1)
print_df["LSRs"] = print_df.apply(lambda x : f"{int(x.lsr_hectares_burned):,} ha ({x.lsr_percent_burned:0.1f}%)", axis=1)
print_df["Late-successional and old-growth forests in LSRs"] = print_df.apply(lambda x : f"{int(x.lsr_og_hectares_burned):,} ha ({x.lsr_og_percent_burned:0.1f}%)", axis=1)

print_df = print_df[[
    "Region/ecoregion",
    "LSRs",
    "Late-successional and old-growth forests in LSRs"
]]

In [23]:
print_df

Region/ecoregion              LSRs  \
eco_group                   yr_group                                           
Region                      1985 - 2019  145,480 ha (1.4%)  28,739 ha (1.9%)   
                            2020         177,136 ha (1.7%)  20,591 ha (1.4%)   
                            Total        322,617 ha (3.1%)  49,331 ha (3.3%)   
Coast Range                 1985 - 2019    1,756 ha (0.1%)     635 ha (0.2%)   
                            2020             169 ha (0.0%)       0 ha (0.0%)   
                            Total          1,926 ha (0.1%)     635 ha (0.2%)   
Olympic Peninsula           1985 - 2019      713 ha (0.0%)     334 ha (0.2%)   
                            2020               0 ha (0.0%)       0 ha (0.0%)   
                            Total            714 ha (0.0%)     334 ha (0.2%)   
Oregon Western Cascades     1985 - 2019  112,970 ha (3.9%)  24,235 ha (4.5%)   
                            2020         174,211 ha (6.0%)  18,283 ha (3.4%)   
                            Total        287,182 ha (9.9%)  42,519 ha (7.9%)   
Washington Western Cascades 1985 - 2019   30,038 ha (0.9%)   3,534 ha (0.8%)   
                            2020           2,755 ha (0.1%)   2,307 ha (0.5%)   
                            Total         32,794 ha (1.0%)   5,841 ha (1.2%)   

                                        Late-successional and old-growth forests in LSRs  
eco_group                   yr_group                                                      
Region                      1985 - 2019                                 10,921 ha (1.7%)  
                            2020                                         9,232 ha (1.5%)  
                            Total                                       20,154 ha (3.2%)  
Coast Range                 1985 - 2019                                    285 ha (0.2%)  
                            2020                                             0 ha (0.0%)  
                            Total                                          286 ha (0.2%)  
Olympic Peninsula           1985 - 2019                                    147 ha (0.2%)  
                            2020                                             0 ha (0.0%)  
                            Total                                          147 ha (0.2%)  
Oregon Western Cascades     1985 - 2019                                  9,899 ha (3.9%)  
                            2020                                         8,057 ha (3.7%)  
                            Total                                       17,956 ha (7.0%)  
Washington Western Cascades 1985 - 2019                                    589 ha (0.3%)  
                            2020                                         1,174 ha (0.7%)  
                            Total                                        1,763 ha (1.0%)